# 기본 라이브러리 로드

In [1]:
import time
import pandas as pd
import numpy as np
import json

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings(action = 'ignore')


# 함수

In [2]:
def try_none_text(x):
    try:
        return x.text
    except:
        return 'NA'
        

In [3]:
def try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        return wd.find_element_by_class_name(class_name).text
    except NoSuchElementException:
        print(col, end=' ')
        return 'NA'
        

In [4]:
def strong_try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        temps = wd.find_elements_by_class_name(class_name)
        for temp in temps:
            if temp.text.startswith('편의'):
                return temp.text.split('\n')[1]
        else:
            return 'NA'
    except NoSuchElementException:
        print(col, end='')
        return 'NA'
        

In [5]:
def review_dict(s):
    review_dict = {}
#     print(s.split('평균 별점'))
    s = s.split('평균 별점')
    review_dict['평균_별점'] = s[-1].strip()
    s = s[0].split('사진')
    if len(s) >= 2:
        review_dict['사진'] = int(s[-1])
        review_dict['리뷰'] = s[0].split()[-1]
    else:
        review_dict['사진'] = 'NA'
        review_dict['리뷰'] = s[0].split()[-1]
# print(s)
    return review_dict

## 지역구 함수

In [6]:
dong_list = ['종로구 청운효자동', '종로구 사직동', '종로구 삼청동', '종로구 부암동', '종로구 평창동', '종로구 무악동',
       '종로구 교남동', '종로구 가회동', '종로구 종로1.2.3.4가동', '종로구 종로5.6가동', '종로구 이화동',
       '종로구 혜화동', '종로구 창신제1동', '종로구 창신제2동', '종로구 창신제3동', '종로구 숭인제1동',
       '종로구 숭인제2동', '중구 소공동', '중구 회현동', '중구 명동', '중구 필동', '중구 장충동',
       '중구 광희동', '중구 을지로동', '중구 신당동', '중구 다산동', '중구 약수동', '중구 청구동',
       '중구 신당제5동', '중구 동화동', '중구 황학동', '중구 중림동', '용산구 후암동', '용산구 용산2가동',
       '용산구 남영동', '용산구 청파동', '용산구 원효로제1동', '용산구 원효로제2동', '용산구 효창동',
       '용산구 용문동', '용산구 한강로동', '용산구 이촌제1동', '용산구 이촌제2동', '용산구 이태원제1동',
       '용산구 이태원제2동', '용산구 한남동', '용산구 서빙고동', '용산구 보광동', '성동구 왕십리제2동',
       '성동구 왕십리도선동', '성동구 마장동', '성동구 사근동', '성동구 행당제1동', '성동구 행당제2동',
       '성동구 응봉동', '성동구 금호1가동', '성동구 금호2.3가동', '성동구 금호4가동', '성동구 옥수동',
       '성동구 성수1가제1동', '성동구 성수1가제2동', '성동구 성수2가제1동', '성동구 성수2가제3동',
       '성동구 송정동', '성동구 용답동', '광진구 화양동', '광진구 군자동', '광진구 중곡제1동',
       '광진구 중곡제2동', '광진구 중곡제3동', '광진구 중곡제4동', '광진구 능동', '광진구 광장동',
       '광진구 자양제1동', '광진구 자양제2동', '광진구 자양제3동', '광진구 자양제4동', '광진구 구의제1동',
       '광진구 구의제2동', '광진구 구의제3동', '동대문구 용신동', '동대문구 제기동', '동대문구 전농제1동',
       '동대문구 전농제2동', '동대문구 답십리제1동', '동대문구 답십리제2동', '동대문구 장안제1동',
       '동대문구 장안제2동', '동대문구 청량리동', '동대문구 회기동', '동대문구 휘경제1동', '동대문구 휘경제2동',
       '동대문구 이문제1동', '동대문구 이문제2동', '중랑구 면목제2동', '중랑구 면목제4동', '중랑구 면목제5동',
       '중랑구 면목본동', '중랑구 면목제7동', '중랑구 면목제3.8동', '중랑구 상봉제1동', '중랑구 상봉제2동',
       '중랑구 중화제1동', '중랑구 중화제2동', '중랑구 묵제1동', '중랑구 묵제2동', '중랑구 망우본동',
       '중랑구 망우제3동', '중랑구 신내1동', '중랑구 신내2동', '성북구 성북동', '성북구 삼선동',
       '성북구 동선동', '성북구 돈암제1동', '성북구 돈암제2동', '성북구 안암동', '성북구 보문동',
       '성북구 정릉제1동', '성북구 정릉제2동', '성북구 정릉제3동', '성북구 정릉제4동', '성북구 길음제1동',
       '성북구 길음제2동', '성북구 종암동', '성북구 월곡제1동', '성북구 월곡제2동', '성북구 장위제1동',
       '성북구 장위제2동', '성북구 장위제3동', '성북구 석관동', '강북구 삼양동', '강북구 미아동',
       '강북구 송중동', '강북구 송천동', '강북구 삼각산동', '강북구 번1동', '강북구 번2동', '강북구 번3동',
       '강북구 수유1동', '강북구 수유2동', '강북구 수유3동', '강북구 우이동', '강북구 인수동',
       '도봉구 창제1동', '도봉구 창제2동', '도봉구 창제3동', '도봉구 창제4동', '도봉구 창제5동',
       '도봉구 도봉제1동', '도봉구 도봉제2동', '도봉구 쌍문제1동', '도봉구 쌍문제2동', '도봉구 쌍문제3동',
       '도봉구 쌍문제4동', '도봉구 방학제1동', '도봉구 방학제2동', '도봉구 방학제3동', '노원구 월계1동',
       '노원구 월계2동', '노원구 월계3동', '노원구 공릉1동', '노원구 공릉2동', '노원구 하계1동',
       '노원구 하계2동', '노원구 중계본동', '노원구 중계1동', '노원구 중계4동', '노원구 중계2.3동',
       '노원구 상계1동', '노원구 상계2동', '노원구 상계3.4동', '노원구 상계5동', '노원구 상계6.7동',
       '노원구 상계8동', '노원구 상계9동', '노원구 상계10동', '은평구 녹번동', '은평구 불광제1동',
       '은평구 불광제2동', '은평구 갈현제1동', '은평구 갈현제2동', '은평구 구산동', '은평구 대조동',
       '은평구 응암제1동', '은평구 응암제2동', '은평구 응암제3동', '은평구 역촌동', '은평구 신사제1동',
       '은평구 신사제2동', '은평구 증산동', '은평구 수색동', '은평구 진관동', '서대문구 천연동',
       '서대문구 북아현동', '서대문구 충현동', '서대문구 신촌동', '서대문구 연희동', '서대문구 홍제제1동',
       '서대문구 홍제제3동', '서대문구 홍제제2동', '서대문구 홍은제1동', '서대문구 홍은제2동',
       '서대문구 남가좌제1동', '서대문구 남가좌제2동', '서대문구 북가좌제1동', '서대문구 북가좌제2동',
       '마포구 아현동', '마포구 공덕동', '마포구 도화동', '마포구 용강동', '마포구 대흥동', '마포구 염리동',
       '마포구 신수동', '마포구 서강동', '마포구 서교동', '마포구 합정동', '마포구 망원제1동',
       '마포구 망원제2동', '마포구 연남동', '마포구 성산제1동', '마포구 성산제2동', '마포구 상암동',
       '양천구 목1동', '양천구 목2동', '양천구 목3동', '양천구 목4동', '양천구 목5동', '양천구 신월1동',
       '양천구 신월2동', '양천구 신월3동', '양천구 신월4동', '양천구 신월5동', '양천구 신월6동',
       '양천구 신월7동', '양천구 신정1동', '양천구 신정2동', '양천구 신정3동', '양천구 신정4동',
       '양천구 신정6동', '양천구 신정7동', '강서구 염창동', '강서구 등촌제1동', '강서구 등촌제2동',
       '강서구 등촌제3동', '강서구 화곡제1동', '강서구 화곡제2동', '강서구 화곡제3동', '강서구 화곡제4동',
       '강서구 화곡본동', '강서구 화곡제6동', '강서구 화곡제8동', '강서구 가양제1동', '강서구 가양제2동',
       '강서구 가양제3동', '강서구 발산제1동', '강서구 우장산동', '강서구 공항동', '강서구 방화제1동',
       '강서구 방화제2동', '강서구 방화제3동', '구로구 신도림동', '구로구 구로제1동', '구로구 구로제2동',
       '구로구 구로제3동', '구로구 구로제4동', '구로구 구로제5동', '구로구 가리봉동', '구로구 고척제1동',
       '구로구 고척제2동', '구로구 개봉제1동', '구로구 개봉제2동', '구로구 개봉제3동', '구로구 오류제1동',
       '구로구 오류제2동', '구로구 수궁동', '구로구 항동', '금천구 가산동', '금천구 독산제1동',
       '금천구 독산제2동', '금천구 독산제3동', '금천구 독산제4동', '금천구 시흥제1동', '금천구 시흥제2동',
       '금천구 시흥제3동', '금천구 시흥제4동', '금천구 시흥제5동', '영등포구 영등포본동', '영등포구 영등포동',
       '영등포구 여의동', '영등포구 당산제1동', '영등포구 당산제2동', '영등포구 도림동', '영등포구 문래동',
       '영등포구 양평제1동', '영등포구 양평제2동', '영등포구 신길제1동', '영등포구 신길제3동',
       '영등포구 신길제4동', '영등포구 신길제5동', '영등포구 신길제6동', '영등포구 신길제7동',
       '영등포구 대림제1동', '영등포구 대림제2동', '영등포구 대림제3동', '동작구 노량진제1동',
       '동작구 노량진제2동', '동작구 상도제1동', '동작구 상도제2동', '동작구 상도제3동', '동작구 상도제4동',
       '동작구 흑석동', '동작구 사당제1동', '동작구 사당제2동', '동작구 사당제3동', '동작구 사당제4동',
       '동작구 사당제5동', '동작구 대방동', '동작구 신대방제1동', '동작구 신대방제2동', '관악구 보라매동',
       '관악구 청림동', '관악구 성현동', '관악구 행운동', '관악구 낙성대동', '관악구 청룡동', '관악구 은천동',
       '관악구 중앙동', '관악구 인헌동', '관악구 남현동', '관악구 서원동', '관악구 신원동', '관악구 서림동',
       '관악구 신사동', '관악구 신림동', '관악구 난향동', '관악구 조원동', '관악구 대학동', '관악구 삼성동',
       '관악구 미성동', '관악구 난곡동', '서초구 서초1동', '서초구 서초2동', '서초구 서초3동',
       '서초구 서초4동', '서초구 잠원동', '서초구 반포본동', '서초구 반포1동', '서초구 반포2동',
       '서초구 반포3동', '서초구 반포4동', '서초구 방배본동', '서초구 방배1동', '서초구 방배2동',
       '서초구 방배3동', '서초구 방배4동', '서초구 양재1동', '서초구 양재2동', '서초구 내곡동',
       '강남구 신사동', '강남구 논현1동', '강남구 논현2동', '강남구 압구정동', '강남구 청담동',
       '강남구 삼성1동', '강남구 삼성2동', '강남구 대치1동', '강남구 대치2동', '강남구 대치4동',
       '강남구 역삼1동', '강남구 역삼2동', '강남구 도곡1동', '강남구 도곡2동', '강남구 개포1동',
       '강남구 개포2동', '강남구 개포4동', '강남구 세곡동', '강남구 일원본동', '강남구 일원1동',
       '강남구 일원2동', '강남구 수서동', '송파구 풍납1동', '송파구 풍납2동', '송파구 거여1동',
       '송파구 거여2동', '송파구 마천1동', '송파구 마천2동', '송파구 방이1동', '송파구 방이2동',
       '송파구 오륜동', '송파구 오금동', '송파구 송파1동', '송파구 송파2동', '송파구 석촌동', '송파구 삼전동',
       '송파구 가락본동', '송파구 가락1동', '송파구 가락2동', '송파구 문정1동', '송파구 문정2동',
       '송파구 장지동', '송파구 위례동', '송파구 잠실본동', '송파구 잠실2동', '송파구 잠실3동',
       '송파구 잠실4동', '송파구 잠실6동', '송파구 잠실7동', '강동구 강일동', '강동구 상일제1동',
       '강동구 상일제2동', '강동구 명일제1동', '강동구 명일제2동', '강동구 고덕제1동', '강동구 고덕제2동',
       '강동구 암사제1동', '강동구 암사제2동', '강동구 암사제3동', '강동구 천호제1동', '강동구 천호제2동',
       '강동구 천호제3동', '강동구 성내제1동', '강동구 성내제2동', '강동구 성내제3동', '강동구 길동',
       '강동구 둔촌제1동', '강동구 둔촌제2동']


In [7]:
from collections import defaultdict
dong_dict = defaultdict(list)
for 구, 동 in [x.split() for x in dong_list]:
    dong_dict[구].append(동)
# dong_dict

In [8]:
dong_key_list = list(dong_dict.keys())
dong_key_list.sort()
# dong_key_list

In [9]:
for 구이름 in dong_key_list:
    dong_dict[구이름] = sorted(dong_dict[구이름])
#     print(dong_dict[구이름])
    

In [10]:
for 구이름 in dong_key_list:
    for 동이름 in dong_dict[구]:
#         print(구이름, 동이름)
        pass

In [11]:
import os
def check_file(file_path, file_name):
    return os.path.isfile(file_path+file_name)

In [12]:
def scroll_down(wd,SCROLL_PAUSE_SEC = 1):

    # 스크롤 높이 가져옴
    last_height = wd.execute_script("return document.body.scrollHeight")
#     print(last_height)
    while True:
        # 끝까지 스크롤 다운
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = wd.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

## json to list

In [13]:
def restNo_script_to_list(wd):
    restNo_list=[]
    script_text = wd.find_element_by_xpath('/html/body/script[3]')

    text1 = script_text.get_attribute('text')
    text1 = text1.split('__APOLLO_STATE__ ')[1]
    text1 = text1.split('[')[1]
    text1 = text1.split(']')[0]
    text1 = '{ "result":[ ' + text1 + ' ]}'
    jsonObj = json.loads(text1)
    
    print(len(jsonObj['result']),end='개완료#')
    for code_num in jsonObj['result']:
        restNo_list.append(str(code_num['id'].split(':')[1]))
    return restNo_list

## 가게코드 가져오기

In [14]:
def restaurant_code_store_scraping( 구이름, 동이름, 업종, slow=1):
    options = webdriver.ChromeOptions()

    # options.add_argument("--start-maximized")
    wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
    wd.implicitly_wait(3)



    pcmap_code_url = 'https://pcmap.place.naver.com/restaurant/list?query='

    search_url_detail = pcmap_code_url + 구이름 +'%20'+ 동이름 + '%20' + 업종
    code_file = '가게코드_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
    code_path = 'data/코드/'
    
    code_df = pd.DataFrame()
    store_idx= 0
    wd.get(search_url_detail)
    time.sleep(3*slow)

    restNo_list=[]
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    print(f'@@@{구이름} {동이름} {업종} 식당 코드 수집을 시작합니다~~ 후비고~~')

    last_num = wd.find_elements_by_class_name('_2tk2s ')[-1].text
    current_num = wd.find_element_by_class_name('_5vmWW').text
    
    restNo_list += restNo_script_to_list(wd)

    code_df = pd.DataFrame(data={'rest_No':list(set(restNo_list))})
    code_df.to_csv(code_path + code_file, index=False)
    print('\n')
    print(구이름,동이름,업종, f'가게 {code_df.shape[0]}개 식당코드 수집 완료' )
    print('\n','########################################################')
    wd.close()


    

In [15]:
# 구이름 = '중랑구'
# 동이름 = '망우본동'
# 업종 = '식당'



In [16]:
# restaurant_code_store_scraping(구이름, 동이름, 업종)

In [17]:
def get_review(review_block, name, code_no):
    reviewer_info = review_block.find_element_by_class_name('ql4ZC').text.replace('\n', ' ')
    reviewer_info = review_dict(reviewer_info)
#             print('reviewer_info',reviewer_info)

    r_score = review_block.find_element_by_class_name('_2tObC').text
#             print('r_score',r_score)

    r_info_list = review_block.find_element_by_class_name('ZvQ8X').find_elements_by_class_name('_3WqoL')
#     print(len(r_info_list))
    r_date = r_info_list[0].text
    r_times = r_info_list[1].text
    proving = r_info_list[2].text
#             print('r_date',r_date)
#             print('r_times',r_times)
#             print('proving',proving)

    r_comment = try_none_text_by_class_name(review_block,'PVBo8','').replace('접기','r')            
#             print('#### r_comment ####')
#             print(r_comment)

    owner_reply = try_none_text_by_class_name(review_block,'Dp7-j','o')
#             print('####owner_reply###')
#             print(owner_reply)

#             print('##################')
    return name, code_no, reviewer_info, r_score, r_date, r_times, proving, r_comment, owner_reply

In [18]:
def naver_review_scrapping( 구이름, 동이름, 업종, slow= 1):
    start_time = time.time()

    options = webdriver.ChromeOptions()

    # options.add_argument("--start-maximized")
    wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
    wd.implicitly_wait(1*slow)
    
    review_path = 'data/리뷰/'
    code_path = 'data/코드/'
    
    code_file = '가게코드_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
    review_file = '리뷰정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
    
    code_df = pd.read_csv(code_path+code_file, dtype='str')
    
    print(f'{구이름} {동이름} {업종} 의 리뷰 스크래핑을 시작합니다.')

    review_idx = 0
    review_df = pd.DataFrame(columns=("name", "code_no", "reviewer_info", "r_score", "r_date", 'r_times', 'proving', 'r_comments', 'owner_reply'))

    code_no = code_df.loc[3].values[0]
    for store_idx, code_no_ in code_df.iterrows():
        code_no = code_no_.values[0]
    
        place_url = 'https://pcmap.place.naver.com/restaurant/'+str(code_no)+'/review/visitor'
        wd.get(place_url)
        time.sleep(1*slow)
        scroll_down(wd,2)
        cnt_idx = 0
        error_idx = 0 

        name = wd.find_element_by_class_name('_3XamX').text
        print(f'{store_idx}번째 {name} 시작', end='\t')
#         print('name',name)
#         print('code_no',code_no)
        scroll_start_time=time.time()
        try:
            while (time.time() - scroll_start_time) < 10: # 20권장 
#                 print(time.time() - scroll_start_time)
                wd.find_element_by_class_name('_3iTUo').click()
                time.sleep(0.3*slow)
        except NoSuchElementException:
            print(f'{int(time.time() - scroll_start_time)}초 소요','로딩완료', end='\t')
        except StaleElementReferenceException :
            print("S_E_Exc")
            pass
        except:
            pass
        
        
        for o_bnt in wd.find_elements_by_class_name('dX2wL'):
            try:
                o_bnt.find_element_by_xpath('..').find_element_by_xpath('..').click()
                print('펼', end='')
            except:
                pass
        else:
            print('펼치기 완료')
        review_block_list = wd.find_elements_by_class_name('_2Cv-r')
#         print(len(review_block_list))
        for review_block in review_block_list:

            try:
                review_df.loc[review_idx] = get_review(review_block, name, code_no)
                review_idx += 1
                cnt_idx += 1
                print('#',end='')
            except:
                print('!!',end='')
                error_idx += 1
                if (error_idx > 10) and (cnt_idx/error_idx < 1):
                    break
                pass
#             #############################
#             if cnt_idx > 5:
#                 break
        else:
            print(f'    {name} {cnt_idx}개 완료.')
            
        if (store_idx+1)%30 == 0:
            wd.close()
            time.sleep(5*slow)
            wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
            wd.implicitly_wait(1*slow)
            print('reboot chrome')
        
    review_df.to_csv(review_path+review_file, index=False)
    print(f'{구이름} {동이름} {업종} 의 리뷰 스크래핑을 완료했습니다.')
    print(f'{review_idx} 개 수집완료, {time.time()-start_time}')
    wd.close()

naver_review_scrapping( 구이름, 동이름, 업종, slow= 1)

# 스크래핑 수행

In [19]:
dong_key_list # 사용 가능 구이름

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

* 준혁 : 

'강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
    
    
    
* 승순 :    
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 
 
 
* 세원 :
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구'

In [20]:
dong_dict['중랑구'] 

['망우본동',
 '망우제3동',
 '면목본동',
 '면목제2동',
 '면목제3.8동',
 '면목제4동',
 '면목제5동',
 '면목제7동',
 '묵제1동',
 '묵제2동',
 '상봉제1동',
 '상봉제2동',
 '신내1동',
 '신내2동',
 '중화제1동',
 '중화제2동']

In [21]:
dong_dict['중랑구'] = ['망우본동',
 '망우제3동',
 '면목본동',
 '면목제2동',
 '면목제4동',
 '면목제5동',
 '면목제7동',
 '묵제1동',
 '묵제2동',
 '상봉제1동',
 '상봉제2동',
 '신내1동',
 '신내2동',
 '중화제1동',
 '중화제2동']

In [22]:
dong_dict['중랑구']

['망우본동',
 '망우제3동',
 '면목본동',
 '면목제2동',
 '면목제4동',
 '면목제5동',
 '면목제7동',
 '묵제1동',
 '묵제2동',
 '상봉제1동',
 '상봉제2동',
 '신내1동',
 '신내2동',
 '중화제1동',
 '중화제2동']

In [23]:
code_path = 'data/코드/'
review_path = 'data/리뷰/'
store_path = 'data/가게/'

In [24]:
업종 = '식당' # 까페
구이름 = '중랑구'
동이름_list = dong_dict[구이름]
# 동이름_list = ['망우본동', '망우제3동']

code_path = 'data/코드/'
review_path = 'data/리뷰/'
store_path = 'data/가게/'

code_file = '가게코드_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
review_file = '리뷰정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
store_file = '가게정보' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'

for 동이름 in 동이름_list:
    code_file = '가게코드_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
    if check_file(file_path=code_path, file_name=code_file):
        print(f'{code_file}이 있습니다. 다음 지역으로 이동합니다')
        pass
    else: restaurant_code_store_scraping(구이름=구이름, 동이름=동이름, 업종=업종)

## 리뷰 수집

In [25]:
할당_list = ['성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']

for 구이름 in 할당_list[::-1]:
    동이름_list = dong_dict[구이름]        
    for 동이름 in 동이름_list:
        review_file = '리뷰정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
        if check_file(file_path=review_path, file_name=review_file):
            print(f'{code_file}이 있습니다. 다음 지역으로 이동합니다')
            pass
        else: naver_review_scrapping(구이름=구이름, 동이름=동이름, 업종=업종)

In [26]:
할당_list = ['성북구', '송파구', '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구']

for 구이름 in 할당_list[::-1]:
    동이름_list = dong_dict[구이름]        
    for 동이름 in 동이름_list:
        review_file = '리뷰정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
        if check_file(file_path=review_path, file_name=review_file):
            print(f'{code_file}이 있습니다. 다음 지역으로 이동합니다')
            pass
        else: naver_review_scrapping(구이름=구이름, 동이름=동이름, 업종=업종)

In [ ]:
while True:
    try:
        for 구이름 in 할당_list[::-1]:
            동이름_list = dong_dict[구이름]        
            for 동이름 in 동이름_list:
                review_file = '리뷰정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
                if check_file(file_path=review_path, file_name=review_file):
                    print(f'{code_file}이 있습니다. 다음 지역으로 이동합니다')
                    pass
                else: naver_review_scrapping(구이름=구이름, 동이름=동이름, 업종=업종)
    except:
        for 구이름 in 할당_list[::-1]:
            동이름_list = dong_dict[구이름]        
            for 동이름 in 동이름_list:
                review_file = '리뷰정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
                if check_file(file_path=review_path, file_name=review_file):
                    print(f'{code_file}이 있습니다. 다음 지역으로 이동합니다')
                    pass
                else: naver_review_scrapping(구이름=구이름, 동이름=동이름, 업종=업종)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


가게코드_중랑구_천호제3동_식당.csv이 있습니다. 다음 지역으로 이동합니다
중랑구 망우제3동 식당 의 리뷰 스크래핑을 시작합니다.
0번째 용마해장국 시작	9초 소요 로딩완료	펼펼펼펼펼펼펼펼치기 완료
o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #    용마해장국 229개 완료.
1번째 홍이네떡볶이 시작	4초 소요 로딩완료	펼펼펼펼펼펼펼치기 완료
o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o

o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #    써브웨이 망우점 320개 완료.
17번째 메종드그릭 시

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


reboot chrome
30번째 무쏘 망우점 시작	8초 소요 로딩완료	펼펼펼펼펼펼펼펼펼펼펼펼펼펼펼펼치기 완료
o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #    무쏘 망우점 105개 완료.
31번째 소중한식당 시작	2초 소요 로딩완료	펼펼펼펼펼펼펼펼펼펼펼치기 완료
o #o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #    소중한식당 56개 완료.
32번째 잉꼬네떡볶이 시작	1초 소요 로딩완료	펼치기 완료
o #o #o #o #o #o #    잉꼬네떡볶이 6개 완료.
33번째 쿠미카페 시작	2초 소요 로딩완료	펼치기 완료
o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #    쿠미카페 59개 완료.
34번째 알로이 짱 시작	1초 소요 로딩완료	펼펼펼펼펼펼펼치기 완료
o #o #o #o #o #o # o #o #o #o #o #o #o #o

...

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


reboot chrome
30번째 금화왕돈까스 용마산역점 시작	8초 소요 로딩완료	펼펼펼펼펼펼펼펼펼펼펼펼펼펼펼치기 완료
o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o ##o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o ##o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #    금화왕돈까스 용마산역점 212개 완료.
31번째 루앙의부엉이 시작	펼펼펼펼펼펼펼펼펼펼펼펼펼펼펼펼펼펼치기 완료
o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


중랑구 면목제5동 식당 의 리뷰 스크래핑을 시작합니다.
0번째 사가정클라쓰 시작	2초 소요 로딩완료	펼펼펼치기 완료
o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #    사가정클라쓰 50개 완료.
1번째 팡그루 시작	6초 소요 로딩완료	펼펼펼펼펼펼펼치기 완료
o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o # o #    팡그루 135개 완료.
2번째 뱃놈과백정 사가정점 시작	5초 소요 로딩완료	펼펼펼치기 완료
o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o

o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o # o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #    버거메이슨 48개 완료.
25번째 이차돌 면목역점 시작	펼펼펼펼펼펼펼펼펼치기 완료
o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #    이차돌 면목역점 230개 완료.
26번째 찡어찡어 시작	3초 소요 로딩완료	펼펼펼펼펼치기 완료
o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o #o